In [7]:
# 導入必要的套件
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableLambda
from langchain_ollama import ChatOllama

# 建立 Ollama 模型
"""
ChatOllama 支援聊天提示和多輪對話，比較適合多輪交互；
而 OllamaLLM 主要用於單輸入單輸出，較簡單的應用。
語法和功能不同，ChatOllama 比較適合現在這種聊天場景。
"""
model = ChatOllama(
    model="gemma3:4b",
    temperature=0.7
)


In [8]:
# 定義主要的提示模板 - 餐廳評論分析
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個專業的餐廳評論家，擅長分析餐廳的優缺點。"),
        ("human", "請分析這家餐廳：{restaurant_name}"),
    ]
)

In [9]:
pros_template = ChatPromptTemplate.from_messages(
        [
            ("system", "你是一個專業的餐廳評論家，專門分析餐廳的優點。"),
            ("human", "根據餐廳資訊：{restaurant_info}，請列出這家餐廳的優點。"),
        ]
    )

# 使用 LCEL 簡化分支鏈的建立
# 優點分析分支
pros_branch_chain = (
    pros_template 
    | model 
    | StrOutputParser()
)

In [10]:
cons_template = ChatPromptTemplate.from_messages(
        [
            ("system", "你是一個專業的餐廳評論家，專門分析餐廳的缺點。"),
            ("human", "根據餐廳資訊：{restaurant_info}，請列出這家餐廳的缺點。"),
        ]
    )

# 缺點分析分支
cons_branch_chain = (
    cons_template
    | model
    | StrOutputParser()
)

In [11]:
chain = (
    prompt_template
    | model
    | StrOutputParser()
    | RunnableLambda(lambda x:{"restaurant_info":x})
    | RunnableParallel(
        pros = pros_branch_chain,
        cons = cons_branch_chain
    )
)

In [12]:
# 執行並行處理鏈 - 測試餐廳評論分析系統
restaurant_info = """
這是一家位於台北市中心的義大利餐廳，提供傳統義式料理。
餐廳環境優雅，服務人員親切，但價格偏高，等待時間較長。
食物品質不錯，但份量偏少。適合約會或特殊場合用餐。
"""

result = chain.invoke({"restaurant_name": restaurant_info})

result

{'pros': '好的，請提供餐廳資訊，我會立即開始分析並列出優點。我會根據您提供的資料，盡可能詳細地評估餐廳的優點，並提供更具體的建議。 請隨時提供，我準備好了！\n',
 'cons': '好的，讓我來為「La Bella Luna」（意為「美麗月亮」）餐廳進行詳細的評論。\n\n**餐廳名稱：** La Bella Luna\n\n**整體評價：** La Bella Luna 位於台北市信義區，以傳統義式菜色為主，提供了一份相對優雅的用餐體驗，但其價格和服務效率方面仍有明顯的提升空間。整體而言，適合追求浪漫氛圍、願意為特殊場合或體驗加價的顧客，但若對價格敏感或對服務有較高期望，則可能稍感失望。\n\n**菜單內容及價格（僅供參考，實際價格可能因季節及特殊活動而異）：**\n\n*   **開胃菜：**\n    *   義式烤麵包（Bruschetta） - NT$380\n    *   魚子醬與奶油的焗烤蝦球 - NT$580\n    *   番茄羅勒沙拉 - NT$420\n*   **義大利麵：**\n    *   經典義式肉醬麵（Spaghetti Bolognese） - NT$680\n    *   海鮮義大利麵（Linguine alle Vongole） - NT$880\n    *   奶油培根義大利麵（Spaghetti Carbonara） - NT$780\n    *   墨魚義大利麵（Spaghetti Nero di Seppia） - NT$720\n*   **海鮮：**\n    *   烤魚配檸檬與香草 - NT$980\n    *   香煎鮭魚配烤蔬菜 - NT$880\n*   **甜點：**\n    *   提拉米蘇 - NT$380\n    *   義式經典芝士蛋糕 - NT$420\n\n**餐廳的特色：** La Bella Luna 突出的特色是其招牌的「月光魚」，一種僅在特定季節捕獲的深海魚，會以期間限定的菜色呈現。餐廳裝飾以復古的義大利海灘風格為主，有著藍白色的色調，搭配一些漁網和貝殼裝飾，營造出輕鬆愜意的氛圍。\n\n**用餐體驗：** 我們在週六晚上 7:30 點餐，用餐人數為四人。服務人員態度親切，但整體服務效率明顯偏慢。點餐後，等待菜餚上桌的時間超過 45 分鐘，而這四道菜都屬